In [ ]:
%load_ext autoreload
%autoreload 2


import os
import sys
import logging

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pvi.models.logistic_regression import LogisticRegressionModel
from pvi.likelihoods.logistic_regression import LogisticRegressionLikelihood

import torch
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

%matplotlib inline
torch.set_default_dtype(torch.float64)

# Construct logistic regression model

In [ ]:
prior_params = {
    "np1": nn.Parameter(torch.tensor([0.0, 0.0, 0.0]),
                        requires_grad=True),
    "np2": nn.Parameter(torch.tensor([-0.05, -0.05, -0.05]).diag_embed(),
                        requires_grad=True)
}

# Initally local likelihood is zero - init at the prior!
current_params = prior_params 

hyperparameters = {
    "D": 2,
    "optimiser_params": {"lr": 1e-4},
    "epochs": 1000,
    "batch_size": 10,
    "num_elbo_samples": 100,
    "num_predictive_samples": 10
}

model = LogisticRegressionModel(nat_params=prior_params, hyperparameters=hyperparameters)

# Set up data

In [ ]:
data_x = torch.tensor([[2, 2], [1, 1], [0, 1], [1, 0], [-0.5, 0.1], 
                       [-1, -1], [-2, -2], [0, -1], [-1, 0], [0.5, 0.1]])
data_y = torch.tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.float)

In [ ]:
def plot_results(data_x, data_y, opt_params):
    x_vals = data_x[:, 0]
    y_vals = data_x[:, 1]
    labels = data_y
    plt.figure()
    plt.grid(b=True)
    plt.scatter(x_vals, y_vals, c = labels)

    w_map = (-2 * opt_params["np2"]).inverse().matmul(opt_params["np1"]).detach()
    plt.arrow(0, 0, w_map[0], w_map[1], head_width=0.1)
    plt.plot([-2, 2], [(w_map[0]/w_map[1])*2, (w_map[0]/w_map[1])*-2])
    plt.show()
    
def plot_training(training_array):
    x_vals = np.arange(1, len(training_array)+1)
    plt.figure()
    plt.plot(x_vals, training_array)
    plt.ylabel('ELBO Loss')
    plt.xlabel('Step')
    plt.show()
     
data = {
    "x": data_x,
    "y": data_y,
}

t = {
    "np1": torch.tensor([0., 0., 0.]),
    "np2": torch.tensor([0., 0., 0.]).diag_embed()
}

In [ ]:
t_new = model.fit(data, t)

In [ ]:
opt_params = model.nat_params
plot_results(data_x, data_y, opt_params)

In [ ]:
plot_training(model._training_curves[0]["elbo"])